In [ ]:
import os
import time
import csv
import glob
import calendar
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# ---------------- CONFIG ----------------
DOWNLOAD_DIR = os.path.abspath("./downloads")  # Folder to save downloaded files
BASE_URL = "https://appv2.loveinstore.com/Home"
MAX_WAIT_SECS = 45
HEADLESS = False

# ✅ --- PUT YOUR LOGIN CREDENTIALS HERE ---
USERNAME = "MISRSPLGT-01"
PASSWORD = "Mis@LIS@2024"
# -----------------------------------------

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# =========================
# CHROME (headless) + downloads
# =========================
chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--ignore-ssl-errors=yes")

prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, MAX_WAIT_SECS)
actions = ActionChains(driver)

# =========================
# HELPER FUNCTIONS
# =========================
def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(locator, timeout=40, attempts=3):
    for attempt in range(attempts):
        el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))
        try:
            el.click()
            return el
        except Exception:
            try:
                js_click(el)
                return el
            except StaleElementReferenceException:
                if attempt == attempts - 1:
                    raise
                time.sleep(0.3)
    raise TimeoutException(f"Could not click {locator}")

def switch_into_frame_containing(locator, timeout=30):
    driver.switch_to.default_content()
    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
        return False
    except TimeoutException:
        pass
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    for fr in frames:
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(fr)
            WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
            return True
        except Exception:
            continue
    driver.switch_to.default_content()
    return False

def wait_for_url_contains(part, timeout=40):
    WebDriverWait(driver, timeout).until(lambda d: part in d.current_url)

def set_date_input(input_el, value_ddmmyyyy):
    driver.execute_script("arguments[0].removeAttribute('readonly');", input_el)
    driver.execute_script("arguments[0].removeAttribute('disabled');", input_el)
    input_el.clear()
    input_el.click()
    input_el.send_keys(value_ddmmyyyy)
    actions.move_by_offset(0, 300).click().perform()

def find_first(locators, timeout_each=20):
    last_err = None
    for loc in locators:
        try:
            return WebDriverWait(driver, timeout_each).until(EC.presence_of_element_located(loc))
        except Exception as e:
            last_err = e
    raise last_err

def wait_for_download(dir_path, timeout=180):
    start = time.time()
    seen = set(glob.glob(os.path.join(dir_path, "*")))
    while time.time() - start < timeout:
        time.sleep(1)
        current = set(glob.glob(os.path.join(dir_path, "*")))
        new_files = list(current - seen)
        if new_files:
            # Wait until file finishes downloading (.crdownload gone)
            while any(f.endswith(".crdownload") for f in new_files):
                time.sleep(1)
            return max(new_files, key=os.path.getmtime)
    return None

# =========================
# SCRIPT START TIME
# =========================
script_start_time = time.time()

# =========================
# MAIN SCRIPT FLOW
# =========================
try:
    print("Opening site…")
    driver.get(BASE_URL)

    # --- LOGIN (Improved & Safe) ---
    print("Logging in…")
    username_field = wait.until(EC.presence_of_element_located((By.ID, "UserName")))
    password_field = wait.until(EC.presence_of_element_located((By.ID, "Password")))
    login_button   = wait.until(EC.element_to_be_clickable((By.ID, "sign")))

    for el in [username_field, password_field]:
        driver.execute_script("arguments[0].removeAttribute('readonly');", el)
        driver.execute_script("arguments[0].removeAttribute('disabled');", el)
        time.sleep(0.2)

    username_field.clear()
    password_field.clear()
    username_field.send_keys(USERNAME)
    time.sleep(0.2)
    password_field.send_keys(PASSWORD)
    time.sleep(0.3)
    login_button.click()

    wait.until(EC.any_of(
        EC.presence_of_element_located((By.XPATH, "//aside|//div[contains(@class,'sidebar')]")),
        EC.presence_of_element_located((By.XPATH, "//a[contains(.,'Logout') or contains(@href,'Logout')]"))
    ))
    print("✅ Logged in successfully.")
    time.sleep(1)

    # --- Navigate: Data Management → Store Action → Primary Shelf Stock ---
    print("Opening Data Management…")
    safe_click((By.XPATH, "//a[contains(@href,'/Admin/DataManagement/Index') and .//span[contains(text(),'Data Management')]]"))

    print("Opening Store Action…")
    safe_click((By.XPATH, "//a[contains(.,'Store Action') and not(contains(@href,'PrimaryShelfStock'))]"))

    print("Opening Primary Shelf Stock…")
    safe_click((By.XPATH, "//a[contains(.,'Primary Shelf Stock')]"))
    wait_for_url_contains("StoreAction", timeout=40)

    target_dropdown = (By.XPATH, "//*[@id='Sdatewise' or @name='Sdatewise' or @id='select2-Sdatewise-container']")
    switch_into_frame_containing(target_dropdown, timeout=30)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located(target_dropdown))

    print("Selecting Datewise option…")
    select2_container = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@id='select2-Sdatewise-container']"))
    )
    select2_container.click()
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[contains(@class,'select2-search__field')]"))
    )
    search_box.clear()
    search_box.send_keys("Datewise")
    time.sleep(0.5)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)

    # =========================
    # DATE RANGE LOOP LOGIC
    # =========================
    today = datetime.today()
    year = today.year
    month = today.month
    yesterday = today - timedelta(days=1)
    last_day = calendar.monthrange(year, month)[1]

    # If today is 11th → only 1–10; if 26th → only 1–25, etc.
    effective_last = min(yesterday.day, last_day)

    # Define all possible date ranges
    ranges = [
        (1, 5),
        (6, 10),
        (11, 15),
        (16, 20),
        (21, 25),
        (26, last_day)
    ]

    # Trim ranges so we don’t go past yesterday
    ranges = [(s, e if e <= effective_last else effective_last) for s, e in ranges if s <= effective_last]

    print(f"📅 Will download for ranges: {ranges}")

    for start_day, end_day in ranges:
        start_date = datetime(year, month, start_day).strftime("%d/%m/%Y")
        end_date = datetime(year, month, end_day).strftime("%d/%m/%Y")
        print(f"🔁 Processing {start_date} → {end_date}")

        from_input = find_first([
            (By.ID, "Sdate"),
            (By.NAME, "DateFrom"),
            (By.XPATH, "//input[@placeholder='From' or @placeholder='Date From']")
        ])
        to_input = find_first([
            (By.ID, "dateto"),
            (By.NAME, "DateTo"),
            (By.XPATH, "//input[@placeholder='To' or @placeholder='Date To']")
        ])

        set_date_input(from_input, start_date)
        set_date_input(to_input, end_date)
        time.sleep(1)

        # =========================
        # FIXED DOWNLOAD/EXPORT CLICK
        # =========================
        print("Triggering download…")
        try:
            # Click 'Download Now' to open dropdown
            download_now_btn = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//a[contains(@class,'dropdown-toggle') and contains(.,'Download Now')]")
                )
            )
            js_click(download_now_btn)  # safer click via JS
            time.sleep(0.5)

            # Click 'Export' inside the visible dropdown
            export_btn = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//div[contains(@class,'dropdown-menu') and contains(@class,'show')]//button[@value='Export']")
                )
            )
            js_click(export_btn)
            time.sleep(1)  # small delay to let download start
        except Exception as e:
            print("⚠️ Could not trigger download/export:", e)

        downloaded = wait_for_download(DOWNLOAD_DIR, timeout=180)
        if downloaded:
            new_name = f"PrimaryShelfStock_{start_date.replace('/','-')}_to_{end_date.replace('/','-')}.csv"
            new_path = os.path.join(DOWNLOAD_DIR, new_name)
            os.rename(downloaded, new_path)
            print(f"✅ Downloaded: {new_name}")
        else:
            print(f"⚠️ No download detected for {start_date} → {end_date}")

    print("🎉 All date ranges processed successfully.")

except Exception as e:
    print("❌ Automation failed:", e)

finally:
    try:
        driver.quit()
    except Exception:
        pass

    script_end_time = time.time()
    total_runtime = script_end_time - script_start_time
    minutes, seconds = divmod(total_runtime, 60)
    print(f"⏱ Total script runtime: {int(minutes)} min {int(seconds)} sec")


Opening site…
Logging in…
✅ Logged in successfully.
Opening Data Management…
Opening Store Action…
Opening Primary Shelf Stock…
Selecting Datewise option…
📅 Will download for ranges: [(1, 5), (6, 10)]
🔁 Processing 01/11/2025 → 05/11/2025
Triggering download…
